In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
import math
from torch.utils.data import Dataset, DataLoader
from sklearn import datasets
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.preprocessing import StandardScaler #para escalar caracteristicas
from sklearn.model_selection import train_test_split # separar mas facil la data de train y test
import pandas as pd
from sklearn.utils import shuffle
from sklearn import preprocessing

## Device config

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Esto garantiza que se ejecutara en GPU si esta disponible
print(device)

cpu


## MNIST Data

In [3]:
df_raw = pd.read_csv('D:/.Memoria/Test/Jupyter/test_example.csv')
columns = list(df_raw.columns)
#  Eliminar los con tiempos de ejecución 0 o muy altos
df_clean = shuffle(df_raw[(df_raw['ql_rt_msec'] > 0) & (df_raw['ql_rt_msec'] < 1e4)])
#df_clean = shuffle(df_raw)

df_clean.to_csv('D:/.Memoria/Test/Jupyter/test_example_clean.csv',index=False)

### HAY QUE CONVERTIR ALGUNAS COLUMNAS Q ESTAN EN PORCENTAJE A FLOAT
df_clean['ql_rt_clocks'] = df_clean['ql_rt_clocks'].apply(lambda x: float(x.strip('%'))/100)
df_clean['ql_same_seg'] = df_clean['ql_same_seg'].apply(lambda x: float(x.strip('%'))/100)
df_clean['ql_same_page'] = df_clean['ql_same_page'].apply(lambda x: float(x.strip('%'))/100)
df_clean['ql_cl_wait_clocks'] = df_clean['ql_cl_wait_clocks'].apply(lambda x: float(x.strip('%'))/100)
df_clean['ql_c_clocks'] = df_clean['ql_c_clocks'].apply(lambda x: float(x.strip('%'))/100)
df_clean['ql_c_cl_wait'] = df_clean['ql_c_cl_wait'].apply(lambda x: float(x.strip('%'))/100)


print(f'shape df_raw: {df_raw.shape}')

df_clean = df_clean[['limit', 'ql_rt_clocks', 'ql_rnd_rows', 'ql_seq_rows', 'ql_same_seg', 'ql_same_page', 'ql_disk_reads', 'ql_spec_disk_reads', 'ql_cl_wait_clocks', 'ql_c_msec', 'ql_c_disk', 'ql_c_clocks', 'ql_cl_messages', 'ql_c_cl_wait','ql_rt_msec']]
print(f'shape df_clean: {df_clean.shape}')


msk = np.random.rand(len(df_clean)) <= 0.8
df_train = df_clean[msk]
df_test = df_clean[~msk]

df_train.to_csv('D:/.Memoria/Test/Jupyter/test_example_clean_train.csv',index=False)
df_test.to_csv('D:/.Memoria/Test/Jupyter/test_example_clean_test.csv',index=False)

#print(f'shape df_train: {df_train.shape}')
#print(f'shape df_test: {df_test.shape}')


## FEATURES - TRAIN Y TEST
X_df_train = df_train[['limit', 'ql_rt_clocks', 'ql_rnd_rows', 'ql_seq_rows', 'ql_same_seg', 'ql_same_page', 'ql_disk_reads', 'ql_spec_disk_reads', 'ql_cl_wait_clocks', 'ql_c_msec', 'ql_c_disk', 'ql_c_clocks', 'ql_cl_messages', 'ql_c_cl_wait']]
X_df_test = df_test[['limit', 'ql_rt_clocks', 'ql_rnd_rows', 'ql_seq_rows', 'ql_same_seg', 'ql_same_page', 'ql_disk_reads', 'ql_spec_disk_reads', 'ql_cl_wait_clocks', 'ql_c_msec', 'ql_c_disk', 'ql_c_clocks', 'ql_cl_messages', 'ql_c_cl_wait']]

X_numpy_train = X_df_train.to_numpy().astype(np.float32)
X_numpy_test = X_df_test.to_numpy().astype(np.float32)

###SOLO PA COMPARAR NO SIRVEN DE NADA DESPUES BORRAR
X_numpy_train_old =  X_numpy_train
X_numpy_test_old = X_numpy_test

######

X_numpy_train = preprocessing.normalize(X_numpy_train)
X_numpy_test = preprocessing.normalize(X_numpy_test)

#X_numpy_train = preprocessing.scale(X)


## TARGETS - TRAIN Y TEST
y_df_train = df_train['ql_rt_msec']
y_df_test = df_test['ql_rt_msec']

y_numpy_train = y_df_train.to_numpy().astype(np.float32)
y_numpy_test = y_df_test.to_numpy().astype(np.float32)


# Pasarlos a Torch. 
X_train = torch.from_numpy(X_numpy_train)
X_test = torch.from_numpy(X_numpy_test)
y_train = torch.from_numpy(y_numpy_train)
y_test = torch.from_numpy(y_numpy_test)
# También pasar los targets de vector fila a vector columna
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)


print("-----------------------")
print(f'shape X_numpy_train: {X_numpy_train.shape}')
print(f'shape X_numpy_test: {X_numpy_test.shape}')
print(f'shape y_numpy_train: {y_numpy_train.shape}')
print(f'shape y_numpy_test: {y_numpy_test.shape}')
print("-----------------------")
print(f'shape X_train: {X_train.shape}')
print(f'shape X_test: {X_test.shape}')
print(f'shape y_train: {y_train.shape}')
print(f'shape y_test: {y_test.shape}')
print("-----------------------")

shape df_raw: (3711, 21)
shape df_clean: (2253, 15)
-----------------------
shape X_numpy_train: (1810, 14)
shape X_numpy_test: (443, 14)
shape y_numpy_train: (1810,)
shape y_numpy_test: (443,)
-----------------------
shape X_train: torch.Size([1810, 14])
shape X_test: torch.Size([443, 14])
shape y_train: torch.Size([1810, 1])
shape y_test: torch.Size([443, 1])
-----------------------


In [4]:
print(X_numpy_train_old)

[[0.00000e+00 1.00000e-02 5.64000e+02 ... 0.00000e+00 0.00000e+00
  0.00000e+00]
 [0.00000e+00 6.20000e-01 2.62600e+03 ... 0.00000e+00 0.00000e+00
  0.00000e+00]
 [0.00000e+00 6.60000e-01 8.23000e+02 ... 0.00000e+00 0.00000e+00
  0.00000e+00]
 ...
 [0.00000e+00 9.50000e-01 4.00000e+00 ... 0.00000e+00 0.00000e+00
  0.00000e+00]
 [0.00000e+00 1.00000e-02 1.74000e+02 ... 9.57169e-01 0.00000e+00
  0.00000e+00]
 [0.00000e+00 7.30000e-01 5.03000e+02 ... 0.00000e+00 0.00000e+00
  0.00000e+00]]


In [5]:
print(X_numpy_train)

[[0.0000000e+00 1.6036898e-05 9.0448111e-01 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 6.7112829e-05 2.8425533e-01 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 2.0092716e-04 2.5055006e-01 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 ...
 [0.0000000e+00 2.0299098e-01 8.5469890e-01 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 5.0514012e-05 8.7894380e-01 ... 4.8350445e-03
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 9.4530216e-05 6.5135203e-02 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]]


## Parametros

In [6]:
n_samples_train, n_features_train = X_train.shape
n_samples_test, n_features_test = X_test.shape
learning_rate = 0.001
hidden_size = 10000
input_size = n_features_train 

## Modelo


In [7]:
class NeuralNet(nn.Module):
    def __init__(self,input_size,hidden_size):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, 1)
        
    # Se aplica el modelo construido arriba en el forward    
    def forward(self,x):
        x=self.l1(x)
        x=F.relu(x)
        x=self.l2(x)
        return x
        
# Definimos el modelo con la clase
model = NeuralNet(input_size,hidden_size).to(device)

## Loss y optimizer

In [8]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

## Training Loop

In [9]:
for epoch in range(100000):
    y_hat = model(X_train)
    loss = criterion(y_hat,y_train)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if (epoch+1)%500==0:
        print(f'epoch: {epoch+1}, loss = {loss.item()}')

epoch: 500, loss = 504454.96875
epoch: 1000, loss = 491001.625
epoch: 1500, loss = 480281.5625
epoch: 2000, loss = 471798.0625
epoch: 2500, loss = 466057.71875
epoch: 3000, loss = 462178.84375
epoch: 3500, loss = 459196.78125
epoch: 4000, loss = 456698.875
epoch: 4500, loss = 454375.28125
epoch: 5000, loss = 452153.625
epoch: 5500, loss = 449987.34375
epoch: 6000, loss = 447947.84375
epoch: 6500, loss = 446006.90625
epoch: 7000, loss = 444139.125
epoch: 7500, loss = 442365.84375
epoch: 8000, loss = 440497.71875
epoch: 8500, loss = 438198.96875
epoch: 9000, loss = 436104.78125
epoch: 9500, loss = 434113.90625
epoch: 10000, loss = 432115.59375
epoch: 10500, loss = 430098.46875
epoch: 11000, loss = 428191.46875
epoch: 11500, loss = 426229.4375
epoch: 12000, loss = 424371.125
epoch: 12500, loss = 422617.75
epoch: 13000, loss = 420888.71875
epoch: 13500, loss = 419130.4375
epoch: 14000, loss = 417473.78125
epoch: 14500, loss = 415792.84375
epoch: 15000, loss = 414197.0
epoch: 15500, loss = 

In [10]:
with torch.no_grad():
    print(X_test.shape)
    print(X_test[0])

torch.Size([443, 14])
tensor([0.0000e+00, 8.5845e-05, 8.3591e-02, 9.9650e-01, 0.0000e+00, 1.5455e-05,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 3.4685e-04, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00])


In [11]:


with torch.no_grad():
    outputs = model(X_test)
    for i in range(0,outputs.shape[0]):
        print(y_test[i],outputs[i])

tensor([839.]) tensor([176.0281])
tensor([2204.]) tensor([198.2200])
tensor([26.]) tensor([4527.6665])
tensor([1.]) tensor([12.7244])
tensor([4.]) tensor([-24.3109])
tensor([3.]) tensor([64.6655])
tensor([1558.]) tensor([217.8521])
tensor([240.]) tensor([43.8811])
tensor([1.]) tensor([95.2469])
tensor([2.]) tensor([175.0770])
tensor([3.]) tensor([-4.8499])
tensor([49.]) tensor([315.7921])
tensor([41.]) tensor([351.1689])
tensor([2.]) tensor([-7.3975])
tensor([8.]) tensor([494.1892])
tensor([65.]) tensor([66.1805])
tensor([1.]) tensor([121.1951])
tensor([8596.]) tensor([222.6588])
tensor([381.]) tensor([360.2102])
tensor([37.]) tensor([210.6599])
tensor([1.]) tensor([-13.3906])
tensor([712.]) tensor([845.6531])
tensor([2.]) tensor([2.0464])
tensor([17.]) tensor([183.3707])
tensor([13.]) tensor([62.1664])
tensor([162.]) tensor([168.9139])
tensor([29.]) tensor([475.5084])
tensor([26.]) tensor([-296.6458])
tensor([274.]) tensor([83.3003])
tensor([2.]) tensor([20.8392])
tensor([33.]) tensor

tensor([2651.]) tensor([1685.8665])
tensor([4.]) tensor([-21.3320])
tensor([3.]) tensor([-3.9547])
tensor([2.]) tensor([-14.8564])
tensor([6.]) tensor([41.9734])
tensor([5.]) tensor([44.6682])
tensor([4.]) tensor([37.9402])
tensor([140.]) tensor([192.4915])
tensor([16.]) tensor([-28.3960])
tensor([1704.]) tensor([184.0295])
tensor([16.]) tensor([455.9210])
tensor([1.]) tensor([-40.0374])
tensor([1532.]) tensor([3691.3772])
tensor([3694.]) tensor([251.3917])
tensor([1.]) tensor([-22.2206])
tensor([2.]) tensor([-5.8857])
tensor([6.]) tensor([154.9349])
tensor([1.]) tensor([-3.6027])
tensor([474.]) tensor([213.7139])
tensor([6.]) tensor([309.8810])
tensor([1.]) tensor([-1.6361])
tensor([3.]) tensor([-1.1993])
tensor([841.]) tensor([3364.8674])
tensor([3.]) tensor([-9.6318])
tensor([1.]) tensor([-40.0935])
tensor([2.]) tensor([47.0781])
tensor([2.]) tensor([-13.5872])
tensor([81.]) tensor([206.4850])
tensor([191.]) tensor([381.2124])
tensor([4.]) tensor([-8.4587])
tensor([4.]) tensor([-6.6

In [ ]:
# SIEMPRE EVITAR CONSIDERAR LOS GRADIENTES
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        # Debemos hacer nuevamente el reshape OJO
        images = images.reshape(-1, 28*28).to(device) #Se envia el GPU si esta disponible...
        labels = labels.to(device)
        outputs = model(images)
        
        # value, index
        _, predictions = torch.max(outputs, 1) # obtener el max posible valor
        n_samples += labels.shape[0] # numero de muestras por lote actual
        n_correct += (predictions == labels).sum().item() #ver cuantas acertaron, si acerto suma 1 si no 0
    # Medir % precision
    acc = 100.0 * n_correct/n_samples
    
    print(f'accuracy = {acc}')
        
        